# <span style='color:green; font-family:Helvetica'> Classifying Pepper and Weeping Willow Trees </span>
### <span style='color:green; font-family:Helvetica'> by DeepSquad </span>

<img src="images_trees/ICON.png" alt="Image" style="width:500px;height:400px;">


## <span style='color:green; font-family:Helvetica'> Data Preparation

In [ ]:
import torchvision.transforms as transforms #provides various image transformation functions
from torchvision.datasets import ImageFolder #class allows us to load images from folders organized by class labels

# Define the transformations to apply to the images
# Compose: his function is used to create a sequence of transformations that will be applied to the images. 
# It takes a list of transformation functions as arguments and applies them in sequence.
# Resize: Many pre-trained models, such as ResNet, VGG, and MobileNet, expect input images of this size
# ToTensor: This transformation converts the images from the PIL Image format to a PyTorch tensor. 
# Tensors are the fundamental data structure used in PyTorch for efficient computation.
# Normalize: normalizes the image tensor -Normalizing the input data to have zero mean and unit variance 
# Helps ensure that the model receives consistent and well-scaled inputs. 

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training dataset
train_set = ImageFolder('images_trees/train', transform=transform)

# Load the test dataset
test_set = ImageFolder('images_trees/test', transform=transform)

## <span style='color:green; font-family:Helvetica'> Model Selection

In [ ]:
import torch #provides various functionalities for deep learning and neural networks
import torchvision.models as models #for popular pre-trained models for computer vision tasks.

# Load a pre-trained model ResNet-50 that has been pre-trained on the ImageNet dataset
model = models.resnet50(pretrained=True) 

# Modify the last fully connected layer to match the number of classes
# This line specifies the number of classes in your classification problem. 
# In this case, you have two classes: pepper tree and willow tree

num_classes = 2
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# modifies the last fully connected layer of the ResNet-50 model to match the number of classes in your classification problem
# torch.nn.Linear - used to replace the existing fully connected layer with a new one that has the desired number of output features
# model.fc.in_features - retrieves the input size of the existing fully connected layer
# num_classes - specifies the desired output size


## <span style='color:green; font-family:Helvetica'> Model Fine-Tuning<span style='color:green; font-family:Helvetica'> 

In [ ]:
#In deep learning models, the requires_grad attribute of a parameter determines whether gradients should be computed and stored for that parameter during the backward pass of training. 
#By default, this attribute is set to True, indicating that the parameter participates in the gradient computation.


# Set requires_grad to False for all parameters except the last layer
# This step is important because we want to freeze the weights of the pre-trained layers and avoid updating them during the fine-tuning process.

for param in model.parameters():
    param.requires_grad = False

# Set requires_grad to True for the parameters of the last layer
# we allow gradients to be computed for these parameters during the backward pass. 
# This step is necessary because we want to train the parameters of the last layer from scratch or fine-tune them to fit the specific classification task.

for param in model.fc.parameters():
    param.requires_grad = True


## <span style='color:green; font-family:Helvetica'> Training

In [ ]:
import torch.optim as optim #provides various optimization algorithms
import torch.nn as nn #contains neural network related functions and classes
from torch.utils.data import DataLoader #provides utilities for data loading and manipulation

# Create data loaders for training and testing - to efficiently load and process the training and testing datasets in batches during the training and evaluation phases
# Loading data in batches helps in utilizing GPU parallelism and reduces memory requirements.
# shuffle=True argument for train_loader ensures that the training data is randomly shuffled before each epoch, which introduces randomness and prevents the model from memorizing the order of the samples.
# shuffle=False keeps the testing data in the original order during evaluation to ensure consistency and allow comparison with the ground truth labels.

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffl
                         e=False)

# Define loss function and optimizer
# The loss function (nn.CrossEntropyLoss()) - to calculate the discrepancy between the predicted outputs and the ground truth labels during training.
# It combines softmax activation and the negative log-likelihood loss.
# optimizer (optim.SGD()) - implements the stochastic gradient descent algorithm and updates the model's parameters based on the computed gradients
# model.parameters() provides the parameters of the model that need to be optimized.
# lr=0.001 sets the learning rate to 0.001,  determines the step size for parameter updates during optimization.
# momentum=0.9 sets the momentum to 0.9,  helps accelerate the optimization process by incorporating information from previous parameter updates.

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
# num_epochs = 10 defines the number of times the model will iterate over the entire training dataset.
# device is determined based on the availability of a CUDA-enabled GPU. It ensures that the model and data are moved to the appropriate device (GPU or CPU) for computation.
# model.to(device) moves the model's parameters to the specified device.

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# The training loop iterates over the specified number of epochs and performs the below steps
# Sets the model in the training mode (model.train()) to enable gradient computation and parameter updates.
# Initializes the running_loss variable to keep track of the cumulative loss for each epoch.
# For each batch of images and labels from the training data --
# Moves the data to the specified device (device) for computation.
# Clears the gradients accumulated in the optimizer (optimizer.zero_grad()).
# Forward passes the images through the model to obtain the predicted outputs.
# Calculates the loss between the predicted outputs and the ground truth labels using the specified loss function.
# Backpropagates the gradients through the model and updates the model's parameters using the optimizer.
# Accumulates the batch loss (loss.item()) to the running_loss.
# Prints the average loss per batch for the current epoch.

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")


## <span style='color:green; font-family:Helvetica'> Model Evaluation

In [ ]:
# defines a function evaluate that is used to evaluate the performance of a trained model on a given data loader
# takes two arguments: model (the trained model) and data_loader (the data loader containing the test dataset).
# model.eval() sets the model to evaluation mode, which disables the gradient calculations and other operations specific to training.
# correct and total are initialized to keep track of the number of correctly classified samples and the total number of samples, respectively.
# torch.no_grad() is a context manager that disables gradient calculations and reduces memory consumption during the evaluation loop.
# The loop iterates over the batches of images and labels from the data_loader.
# images and labels are moved to the device (GPU) if available.
# outputs contains the predicted class probabilities for the images obtained from the model.
# torch.max(outputs.data, 1) finds the maximum value and its corresponding index along the dimension 1, which represents the predicted class labels.
# total is updated by adding the size of the current batch of labels.
# correct is updated by summing the number of correctly predicted labels.

def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
# accuracy is calculated by dividing the number of correctly predicted labels (correct) by the total number of labels (total) and multiplying by 100 to obtain a percentage value.
# The evaluate function is called with the trained model and the test_loader.
# The returned accuracy value is stored in test_accuracy.
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


## <span style='color:green; font-family:Helvetica'> Saving the Model

In [ ]:
# torch.save() is a function provided by PyTorch that allows us to save the state dictionary of an object to a file.
# model.state_dict() returns a dictionary containing the parameters and persistent buffers of the model

torch.save(model.state_dict(), 'model.pt')
print("Model Saved")

## <span style='color:green; font-family:Helvetica'> Streamlit UI

In [ ]:
#Run on command line
#streamlit run tree_classification_server.py
